In [20]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
! pip install toxicityclassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.8 MB/s eta 0:00:0000:0100:01


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
from toxicityclassifier import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
classifier = ToxicityClassificatorV1()

MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

In [21]:
def normalize(text):
    is_toxic = classifier.predict(text)[0]
    if is_toxic:
        return 'Toxic text'
    task_prefix = "Spell correct: "
    if type(text) != list: text = [text]
    encoded = tokenizer(
      [task_prefix + sequence for sequence in text],
      padding="longest",
      max_length=MAX_INPUT,
      truncation=True,
      return_tensors="pt",
    )
    predicts = model.generate(**encoded.to(device))
    return tokenizer.batch_decode(predicts, skip_special_tokens=True)

In [33]:
normalized_text = normalize('Звукавая картв Steinberg UR12')
normalized_text

['Звуковая карта Steinberg UMP12']